In [ ]:
from typing import Optional
from pydantic import BaseModel, Field
from agentics import AG

class Movie(BaseModel):
    movie_name: Optional[str] = None
    description: Optional[str] = None
    year: Optional[int] = None

class Genre(BaseModel):
    genre:Optional[str] = Field(
        None,
        description="return one category,   "
                    "e.g. comedy, drama, ...")


movies = AG.from_csv(
    "data/movies.csv", atype=Movie, max_rows=2
) 

In [ ]:
tweet = AG(atype = Genre, provide_explanations=True)
tweets = await (tweet<<movies)
movies.pretty_print()
tweets.pretty_print()

In [ ]:
from agentics.core.transducible_functions import Transduce, transducible
movie = movies.states[1]
movie.year=1972

@transducible(provide_explanation=True)
async def classify_genre(state:Movie)-> Genre:
    """Classify the genre of the source Movie """
    return Transduce(state)

genre, explanation  = await classify_genre(movie)

print(movie.model_dump_json(indent=2))
print(genre.model_dump_json(indent=2))
print(explanation.model_dump_json(indent=2))


```json
{
  "movie_name": "The Godfather",
  "description": "Spanning the years 1945 to 1955, a chronicle of the fictional Italian-American Corleone crime family. When organized crime family patriarch, Vito Corleone barely survives an attempt on his life, his youngest son, Michael steps in to take care of the would-be killers, launching a campaign of bloody revenge.",
  "release_date": "3-24-1972"
}
{
  "genre": "Crime"
}
{
  "explanation": "The target genre 'Crime' was deduced from the source data. The 'description' explicitly mentions an \"Italian-American Corleone crime family\" and \"organized crime\", which are strong indicators of the crime genre. Additionally, the 'movie_name' \"The Godfather\" is widely recognized as a classic crime film, reinforcing the inference. The 'release_date' does not contribute to genre determination.",
  "relevant_source_attributes": [
    "movie_name",
    "description"
  ],
  "confidence": 0.96
}
```

In [ ]:
{
  "movie_name": "The Godfather",
  "description": "Spanning the years 1945 to 1955, ..."
  "release_date": "3-24-1972"
}
genre='Crime'
{
  "explanation": """The target attribute \"genre\" was deduced from the source data.
                    The \"description\" field explicitly mentions  ... """,
  "relevant_source_attributes": [
    "description",
    "movie_name"
  ],
  "confidence": 0.95
}
```